In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import datetime as dt
import quandl

# Data 

In [3]:
datadir='../data/'
assets=pd.read_csv(datadir+'SP500.txt', comment='#').set_index('Symbol')
#  skyJ_Rgz8mgzTuZJ7Biy
QUANDL={
    ## Get a key (free) from quandl.com and copy it here
    'authtoken':"VvmHMhvuE13zMSZxgCEC",
    'start_date':dt.date(2007, 1, 1),
    'end_date':dt.date(2016, 12, 31)
}
RISK_FREE_SYMBOL = "USDOLLAR"
data={}

#### Download loop

If it stops because of Quandl error codes 503 or 504, try re-running it (it won't download data already downloaded). If Quandl complains about the speed of requests, try adding sleep time.

In [4]:
# Download assets' data
from time import sleep

# Construct a data dictionary: {ticker: pd.DataFrame(price/volume)}
for ticker in assets.index:
    if ticker in data:
        continue
    print('downloading %s from %s to %s' %(ticker, QUANDL['start_date'], QUANDL['end_date']))
    try:
        data[ticker] = quandl.get(assets.Quandlcode[ticker], **QUANDL)
        #sleep(0.1)
    except quandl.NotFoundError:
        print('\tInvalid asset code')

downloading MMM from 2007-01-01 to 2016-12-31
downloading ABT from 2007-01-01 to 2016-12-31
downloading ABBV from 2007-01-01 to 2016-12-31
downloading ACN from 2007-01-01 to 2016-12-31
downloading ACE from 2007-01-01 to 2016-12-31
downloading ATVI from 2007-01-01 to 2016-12-31
downloading ADBE from 2007-01-01 to 2016-12-31
downloading ADT from 2007-01-01 to 2016-12-31
downloading AAP from 2007-01-01 to 2016-12-31
downloading AES from 2007-01-01 to 2016-12-31
downloading AET from 2007-01-01 to 2016-12-31
downloading AFL from 2007-01-01 to 2016-12-31
downloading AMG from 2007-01-01 to 2016-12-31
downloading A from 2007-01-01 to 2016-12-31
downloading GAS from 2007-01-01 to 2016-12-31
downloading APD from 2007-01-01 to 2016-12-31
downloading ARG from 2007-01-01 to 2016-12-31
downloading AKAM from 2007-01-01 to 2016-12-31
downloading AA from 2007-01-01 to 2016-12-31
downloading AGN from 2007-01-01 to 2016-12-31
downloading ALXN from 2007-01-01 to 2016-12-31
downloading ALLE from 2007-01-01

downloading ES from 2007-01-01 to 2016-12-31
downloading EXC from 2007-01-01 to 2016-12-31
downloading EXPE from 2007-01-01 to 2016-12-31
downloading EXPD from 2007-01-01 to 2016-12-31
downloading ESRX from 2007-01-01 to 2016-12-31
downloading XOM from 2007-01-01 to 2016-12-31
downloading FFIV from 2007-01-01 to 2016-12-31
downloading FB from 2007-01-01 to 2016-12-31
downloading FAST from 2007-01-01 to 2016-12-31
downloading FDX from 2007-01-01 to 2016-12-31
downloading FIS from 2007-01-01 to 2016-12-31
downloading FITB from 2007-01-01 to 2016-12-31
downloading FSLR from 2007-01-01 to 2016-12-31
downloading FE from 2007-01-01 to 2016-12-31
downloading FISV from 2007-01-01 to 2016-12-31
downloading FLIR from 2007-01-01 to 2016-12-31
downloading FLS from 2007-01-01 to 2016-12-31
downloading FLR from 2007-01-01 to 2016-12-31
downloading FMC from 2007-01-01 to 2016-12-31
downloading FTI from 2007-01-01 to 2016-12-31
downloading F from 2007-01-01 to 2016-12-31
downloading FOSL from 2007-01-

downloading PFE from 2007-01-01 to 2016-12-31
downloading PCG from 2007-01-01 to 2016-12-31
downloading PM from 2007-01-01 to 2016-12-31
downloading PSX from 2007-01-01 to 2016-12-31
downloading PNW from 2007-01-01 to 2016-12-31
downloading PXD from 2007-01-01 to 2016-12-31
downloading PBI from 2007-01-01 to 2016-12-31
downloading PCL from 2007-01-01 to 2016-12-31
downloading PNC from 2007-01-01 to 2016-12-31
downloading RL from 2007-01-01 to 2016-12-31
downloading PPG from 2007-01-01 to 2016-12-31
downloading PPL from 2007-01-01 to 2016-12-31
downloading PX from 2007-01-01 to 2016-12-31
downloading PCP from 2007-01-01 to 2016-12-31
downloading PCLN from 2007-01-01 to 2016-12-31
downloading PFG from 2007-01-01 to 2016-12-31
downloading PG from 2007-01-01 to 2016-12-31
downloading PGR from 2007-01-01 to 2016-12-31
downloading PLD from 2007-01-01 to 2016-12-31
downloading PRU from 2007-01-01 to 2016-12-31
downloading PEG from 2007-01-01 to 2016-12-31
downloading PSA from 2007-01-01 to 20

#### Computation 

In [41]:
keys=[el for el in assets.index if not el in (set(assets.index)-set(data.keys()))]

def select_first_valid_column(df, columns):
    for column in columns:
        if column in df.columns:
            return df[column]

# extract prices
prices=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Adj. Close", "Close", "Value"])
                                              for k in keys])))

#compute sigmas
open_price=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Open"]) for k in keys])))
close_price=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Close"]) for k in keys])))
sigmas = np.abs(np.log(open_price.astype(float))-np.log(close_price.astype(float)))

# extract volumes
volumes=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Adj. Volume", "Volume"])
                                               for k in keys])))

# fix risk free
prices[RISK_FREE_SYMBOL]=10000*(1 + prices[RISK_FREE_SYMBOL]/(100*250)).cumprod()

#### Filtering 

In [42]:
# filter NaNs - threshold at 2% missing values
bad_assets = prices.columns[prices.isnull().sum()>len(prices)*0.02]
if len(bad_assets):
    print('Assets %s have too many NaNs, removing them' % bad_assets)

prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]

nassets=prices.shape[1]

# days on which many assets have missing values
bad_days1=sigmas.index[sigmas.isnull().sum(1) > nassets*.9]
bad_days2=prices.index[prices.isnull().sum(1) > nassets*.9]
bad_days3=volumes.index[volumes.isnull().sum(1) > nassets*.9]
bad_days=pd.Index(set(bad_days1).union(set(bad_days2)).union(set(bad_days3))).sort_values()
print ("Removing these days from dataset:")
print(pd.DataFrame({'nan price':prices.isnull().sum(1)[bad_days],
                    'nan volumes':volumes.isnull().sum(1)[bad_days],
                    'nan sigmas':sigmas.isnull().sum(1)[bad_days]}))

prices=prices.loc[~prices.index.isin(bad_days)]
sigmas=sigmas.loc[~sigmas.index.isin(bad_days)]
volumes=volumes.loc[~volumes.index.isin(bad_days)]

# extra filtering
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))
prices=prices.fillna(method='ffill')
sigmas=sigmas.fillna(method='ffill')
volumes=volumes.fillna(method='ffill')
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))

Assets Index(['ABBV', 'ACE', 'ADT', 'GAS', 'ARG', 'AA', 'ALLE', 'GOOG', 'ALTR',
       'AVGO', 'BXLT', 'BRCM', 'CVC', 'CAM', 'CPGX', 'CMCSK', 'DLPH', 'DAL',
       'DFS', 'DISCK', 'DG', 'DPS', 'EMC', 'FB', 'GM', 'HCA', 'HCBK', 'GMCR',
       'KMI', 'KHC', 'LYB', 'MNK', 'MPC', 'MHFI', 'MJN', 'WRK', 'KORS', 'NAVI',
       'NWSA', 'NWS', 'NLSN', 'PYPL', 'POM', 'PM', 'PSX', 'PCL', 'PPG', 'PCP',
       'QRVO', 'SNDK', 'SNI', 'SIAL', 'HOT', 'TEL', 'TE', 'TDC', 'TRV', 'TWC',
       'TRIP', 'TYC', 'UA', 'VRSK', 'V', 'XYL', 'ZTS'],
      dtype='object') have too many NaNs, removing them
Removing these days from dataset:
            nan price  nan volumes  nan sigmas
2007-01-02        438          NaN         NaN
2007-04-06        438          NaN         NaN
2010-04-02        438          NaN         NaN
2012-04-06        438          NaN         NaN
2012-10-29        438          NaN         NaN
2015-04-03        438          NaN         NaN
          remaining nan price  remaining nan volumes

#### Save 

In [54]:
# make volumes in dollars
volumes = volumes*prices

# compute returns
returns = (prices.diff()/prices.shift(1)).fillna(method='ffill').iloc[1:]

bad_assets = returns.columns[((-.5>returns).sum()>0)|((returns > 2.).sum()>0)]
if len(bad_assets):
    print('Assets %s have dubious returns, removed' % bad_assets)
    
prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]
returns = returns.loc[:,~returns.columns.isin(bad_assets)]

# remove USDOLLAR
prices = prices.iloc[:,:-1]
sigmas = sigmas.iloc[:,:-1]
volumes = volumes.iloc[:,:-1]


# save data
prices.to_csv(datadir+'prices.csv.gz', compression='gzip', float_format='%.3f')
volumes.to_csv(datadir+'volumes.csv.gz', compression='gzip', float_format='%d')
returns.to_csv(datadir+'returns.csv.gz', compression='gzip', float_format='%.3e')
sigmas.to_csv(datadir+'sigmas.csv.gz', compression='gzip', float_format='%.3e')

Assets Index(['AAL', 'AIG', 'ETFC', 'GGP', 'GNW', 'HIG', 'STT', 'XL'], dtype='object') have dubious returns, removed


# Estimates 

In [55]:
print("Typical variance of returns: %g"%returns.var().mean())

Typical variance of returns: 0.000541023


In [56]:
sigma2_n=0.02
sigma2_r=0.0005

np.random.seed(0)
noise=pd.DataFrame(index=returns.index, columns=returns.columns, 
                   data=np.sqrt(sigma2_n)*np.random.randn(*returns.values.shape))
return_estimate= (returns + noise)*sigma2_r/(sigma2_r+sigma2_n)
return_estimate.USDOLLAR = returns.USDOLLAR

return_estimate.to_csv(datadir+'return_estimate.csv.gz', compression='gzip', float_format='%.3e')

In [57]:
returns

,MMM,ABT,ACN,ATVI,ADBE,AAP,AES,AET,AFL,AMG,...,WYN,WYNN,XEL,XRX,XLNX,YHOO,YUM,ZBH,ZION,USDOLLAR
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-04,-0.003961,0.018991,0.022540,0.008159,0.022545,0.006464,0.000456,-0.017724,-0.011027,-0.010861,...,-0.010126,0.000415,0.000854,-0.000592,0.023719,0.048419,-0.002718,0.022309,0.004463,0.000196
2007-01-05,-0.006799,0.000000,-0.013978,-0.005780,-0.004900,-0.022061,-0.028702,-0.017094,0.001312,-0.008021,...,-0.000620,0.005918,-0.020043,-0.009473,-0.009102,0.033147,-0.008688,-0.000634,-0.003002,0.000197
2007-01-08,0.002196,0.003607,0.014449,-0.000581,-0.004185,0.003427,-0.014071,0.003140,0.000655,0.013187,...,0.000931,0.018889,0.000000,0.016736,0.006263,0.006489,-0.002406,-0.006474,0.001686,0.000198
2007-01-09,0.001160,0.002796,-0.002419,-0.000582,-0.020272,0.008537,-0.040913,-0.015411,0.003709,0.009690,...,0.000930,0.003951,-0.000435,-0.002939,-0.004564,-0.012178,0.001206,0.000639,0.005531,0.000198
2007-01-10,0.002188,0.009259,0.005927,0.020373,-0.010346,0.001411,0.028770,0.002690,0.011957,0.004610,...,0.004644,0.033804,0.005224,0.000000,0.015840,0.040609,0.014625,-0.003320,0.005859,0.000198
2007-01-11,0.010276,-0.000794,-0.004017,0.051341,0.016828,0.025641,-0.016393,0.032439,0.030075,0.008617,...,0.008320,0.014348,-0.001299,0.018868,0.013952,0.017422,0.020061,0.005765,-0.007252,0.000199
2007-01-12,0.009027,0.001986,0.021242,-0.018991,0.002006,-0.005495,0.026471,0.000472,0.009176,0.012165,...,-0.006418,0.005485,-0.008239,-0.015625,0.004047,0.008562,-0.002167,0.002293,-0.003113,0.000198
2007-01-16,0.002520,0.003171,-0.023170,0.019358,0.002252,0.000000,0.007641,0.007792,0.003926,-0.003945,...,-0.003076,0.031199,0.006122,0.011758,-0.024587,-0.005433,0.005679,0.000254,-0.001562,0.000199
2007-01-17,-0.008170,0.018573,0.011321,0.011394,0.003745,0.004144,-0.004739,-0.009606,-0.010704,0.006724,...,-0.012959,-0.018469,0.001304,-0.012202,-0.030992,-0.008194,-0.005979,0.007751,0.000120,0.000200


In [58]:
agree_on_sign=np.sign(returns.iloc[:,:-1]) == np.sign(return_estimate.iloc[:,:-1])
print("Return predictions have the right sign %.1f%% of the times"%
      (100*agree_on_sign.sum().sum()/(agree_on_sign.shape[0]*(agree_on_sign.shape[1]-1))))

Return predictions have the right sign 53.7% of the times


In [59]:
volume_estimate=volumes.rolling(window=10, center=False).mean().dropna()
volume_estimate.to_csv(datadir+'volume_estimate.csv.gz', compression='gzip', float_format='%d')
sigmas_estimate = sigmas.rolling(window=10, center=False).mean().dropna()
sigmas_estimate.to_csv(datadir+'sigma_estimate.csv.gz', compression='gzip', float_format='%.3e')

# Risk model 

In [101]:
start_t="2012-01-01"
end_t="2016-12-31"

first_days_month=\
    pd.date_range(start=returns.index[next(i for (i,el) in enumerate(returns.index >= start_t) if el)-1],
                                 end=returns.index[-1], freq='MS')

# Use PCA to identify the top k factors generating full covariance matrix
k=15
exposures, factor_sigma, idyos = {}, {}, {}

# Every first day in each month in period
for day in first_days_month:
    # Grab asset returns for preceding 730 days, compute covariance of returns then eigenvalues/eigenvectors
    used_returns = returns.loc[(returns.index < day)&
                        (returns.index >= day-pd.Timedelta("730 days"))]
    second_moment=np.cov(used_returns.values.T)
    eival, eivec=np.linalg.eigh(second_moment)
    
    # Factor covariance is diagonal in top eigenvalues
    factor_sigma[day]=pd.DataFrame(data=np.diag(eival[-k:]))
    # Exposures are the top k eigenvectors
    exposures[day]=pd.DataFrame(data=eivec[:,-k:],index=returns.columns)
    # Idiosyncratic variances
    idyos[day]=pd.Series(data=np.diag(eivec[:,:-k]@np.diag(eival[:-k])@eivec[:,:-k].T),index=returns.columns)

pd.concat(factor_sigma.values(), axis=0, keys=factor_sigma.keys()).to_hdf(datadir+'risk_model.h5', 'factor_sigma')
pd.concat(exposures.values(), axis=0, keys=exposures.keys()).to_hdf(datadir+'risk_model.h5', 'exposures')
pd.DataFrame(idyos).T.to_hdf(datadir+'risk_model.h5', 'idyos')

# load with
# store = pd.HDFStore('risk_model.h5')
# store.exposures, etc.